# Script Molde

Cria os moldes a partir dos quais as requisições do CEPESPData serão comparadas

In [23]:
import numpy as np
import pandas as pd
import os
import csv

if not os.path.exists("data/moldes"):
    os.makedirs("data/moldes")

## Molde Vagas

In [24]:
# Readind vagas paths and storing
b_path = "raw_data/arquivos_vagas/unziped/"
paths = {str(ano): (lambda ano: os.listdir(b_path + str(ano)))(ano) for ano in range(1998,2022,4)}

# Defining Headers
headers = ["DATA_GERACAO", "HORA_GERACAO", "ANO_ELEICAO","DESCRICAO_ELEICAO", "SIGLA_UF",
           "SIGLA_UE", "NOME_UE", "CODIGO_CARGO", "DESCRICAO_CARGO", "QTDE_VAGAS"]

In [25]:
data = []
for ano in range(1998,2022,4):
    ano = str(ano)
    print("Reading {}...".format(ano))
    if ano == '2014' or ano == '2018':
        skiprows_use=1
    else:
        skiprows_use=None

    for file in paths[ano]:
        if 'txt' in file or 'csv' in file:
            data.append(pd.read_csv(b_path + ano + "/" + file,
                                sep = ';',
                                encoding='latin_1',
                                header=None,
                                skiprows=skiprows_use))        

Reading 1998...
Reading 2002...
Reading 2006...
Reading 2010...
Reading 2014...
Reading 2018...


In [26]:
for i in range(len(data)):
        if len(data[i].columns) == 10:
            data[i].columns = headers
        else:
            data[i] = data[i].iloc[:,[0,1,2,6,9,10,11,12,13,14]]
            data[i].columns = headers
    
# Bindng all data
data = pd.concat(data)

# Droping obs
data = data[np.in1d(data.DESCRICAO_ELEICAO, ['Eleição Suplementar Governador AM', 'Eleição Suplementar Governador - TO'], invert=True)]

# Droping coluns which will not be used
data = data.iloc[:,2:len(data.columns)].drop(["SIGLA_UF", 'DESCRICAO_ELEICAO'], axis = 1)

# Creating by hand president DataFrame
vagas_presidente = {'ANO_ELEICAO': np.arange(1998,2018,4),
                    'SIGLA_UE': "BR",
                    'CODIGO_CARGO': 1,
                    'QTDE_VAGAS': 1}

In [27]:
data = data.apply(lambda x: x.str.upper() if hasattr(x, 'str') else x, axis=0)

In [28]:
data = data[['ANO_ELEICAO', 'SIGLA_UE', 'CODIGO_CARGO', "QTDE_VAGAS"]]

In [29]:
presidente_df = pd.DataFrame(vagas_presidente)
    
# Binding president and data together

data = pd.concat([data, presidente_df])

data = data.drop_duplicates()

print("Writing data into molde_vagas.csv")

data.to_csv("data/moldes/molde_vagas.csv")

Writing data into molde_vagas.csv


## Molde Seção

In [34]:
data = "çççççç"

In [39]:
data.encode('utf-8').decode('ascii', 'ignore')

''

In [12]:
# Readind vagas paths and storing
b_path = "raw_data/arquivos_secao/unziped/"
paths = {str(ano): (lambda ano: os.listdir(b_path + str(ano)))(ano) for ano in range(1998,2022,4)}


# Defining Headers
headers = ["DATA_GERACAO", "HORA_GERACAO", "ANO_ELEICAO", "NUM_TURNO", "DESCRICAO_ELEICAO",
           "SIGLA_UF", "SIGLA_UE", "CODIGO_MUNICIPIO", "NOME_MUNICIPIO","NUM_ZONA","NUM_SECAO",
           "CODIGO_CARGO", "DESCRICAO_CARGO", "QTD_APTOS", "QTD_COMPARECIMENTO", "QTD_ABSTENCOES", 
           "QT_VOTOS_NOMINAIS", "QT_VOTOS_BRANCOS", "QT_VOTOS_NULOS", "QT_VOTOS_LEGENDA",
           "QT_VOTOS_ANULADOS_APU_SEP"]

In [22]:
data = []
for ano in range(1998,2022,4):
    ano = str(ano)
    print("Parsing data from {}...".format(ano))
    data_spec = []
    for file in paths[ano]:
        if 'txt' in file or 'csv' in file:
            print(file)
            data_spec.append(pd.read_csv(b_path + ano + '/' + file, sep = ';', header=None, encoding="latin-1"))

    data_spec = pd.concat(data_spec)
    data_spec.columns = headers

    data_spec = data_spec.loc[:,['ANO_ELEICAO',
                                 'NUM_TURNO',
                                 'DESCRICAO_ELEICAO',
                                 'SIGLA_UE',
                                 'CODIGO_MUNICIPIO',
                                 'NOME_MUNICIPIO',
                                 'CODIGO_CARGO',
                                 'DESCRICAO_CARGO',
                                 'QTD_APTOS',
                                 'QTD_COMPARECIMENTO']]

    data_spec = data_spec.groupby(['ANO_ELEICAO',
                                   'NUM_TURNO',
                                   'DESCRICAO_ELEICAO',
                                   'SIGLA_UE',
                                   'CODIGO_MUNICIPIO',
                                   'NOME_MUNICIPIO',
                                   'CODIGO_CARGO',
                                   'DESCRICAO_CARGO',])

    data_spec = data_spec.sum().reset_index()

    data.append(data_spec)

data = pd.concat(data)

Parsing data from 1998...
detalhe_votacao_secao_1998_RN.txt
detalhe_votacao_secao_1998_AP.txt
detalhe_votacao_secao_1998_BR.txt
detalhe_votacao_secao_1998_MA.txt
detalhe_votacao_secao_1998_AL.txt
detalhe_votacao_secao_1998_RJ.txt
detalhe_votacao_secao_1998_RS.txt
detalhe_votacao_secao_1998_RR.txt
detalhe_votacao_secao_1998_PE.txt
detalhe_votacao_secao_1998_MT.txt
detalhe_votacao_secao_1998_AC.txt
detalhe_votacao_secao_1998_MG.txt
detalhe_votacao_secao_1998_BA.txt
detalhe_votacao_secao_1998_DF.txt
detalhe_votacao_secao_1998_SE.txt
detalhe_votacao_secao_1998_PI.txt
detalhe_votacao_secao_1998_ZZ.txt
detalhe_votacao_secao_1998_AM.txt
detalhe_votacao_secao_1998_PR.txt
detalhe_votacao_secao_1998_MS.txt
detalhe_votacao_secao_1998_SC.txt
detalhe_votacao_secao_1998_TO.txt
detalhe_votacao_secao_1998_GO.txt
detalhe_votacao_secao_1998_RO.txt
detalhe_votacao_secao_1998_SP.txt
detalhe_votacao_secao_1998_PB.txt
detalhe_votacao_secao_1998_PA.txt
detalhe_votacao_secao_1998_CE.txt
detalhe_votacao_secao_

ParserError: Error tokenizing data. C error: Expected 21 fields in line 34841, saw 93


In [ ]:
for ano in np.arange(2010,1998,-8):
    temp = data.loc[(data.ANO_ELEICAO == ano) & (data.CODIGO_CARGO == 5),["QTD_APTOS", "QTD_COMPARECIMENTO"]]
    temp = temp * 2
    data.loc[(data.ANO_ELEICAO == ano) & (data.CODIGO_CARGO == 5),["QTD_APTOS", "QTD_COMPARECIMENTO"]] = temp

temp = data.loc[(data.ANO_ELEICAO == 2002) & (data.CODIGO_CARGO == 1),["QTD_APTOS", "QTD_COMPARECIMENTO"]]

temp = temp / 2

data.loc[(data.ANO_ELEICAO == 2002) & (data.CODIGO_CARGO == 1),["QTD_APTOS", "QTD_COMPARECIMENTO"]] = temp

data.to_csv("data/moldes/molde_secao.csv")    

In [ ]:
data = create_molde_secao()

In [ ]:
data[data.ANO_ELEICAO == 2002].head()